In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/Machine_Learning_UoA/CS5062_AssessmentII_Dataset.zip

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('./Train.csv')
df_train['Trip'].value_counts()

1793    2822
2949    2659
1808    2149
1928    1815
2886    1709
        ... 
2776     135
2956     130
1582     122
2820     107
2610     104
Name: Trip, Length: 136, dtype: int64

In [ ]:
df_train

,Trip,VehicleSpeed_km_h_,Battery_Status
0,1567,7.76,0
1,1567,4.91,0
2,1567,3.47,0
3,1567,3.47,0
4,1567,0.00,0
...,...,...,...
107080,3263,29.25,1
107081,3263,29.25,1
107082,3263,28.58,1
107083,3263,28.58,1


In [ ]:
trip = df_train['Trip'].value_counts()

## trip ID
demo_list = []
for i, j in trip.items():
    demo_list.append(i)

len(demo_list)

136

In [ ]:
# split the train data by trip ID

splited_df = []
for i in demo_list:
    splited_df.append(df_train.loc[df_train['Trip'] == i])

# 136 dataframes with same ID
len(splited_df)

136

In [ ]:
def generate_xy (splited_dfs, time_step):
    x_train, y_train = [], []

    for df in splited_dfs:
        df_length = df.shape[0]
        train_data = df['VehicleSpeed_km_h_'].values.reshape(-1, 1)
        label_data = df['Battery_Status'].values.reshape(-1, 1)
        for i in range(time_step, df_length):
            x_train.append(train_data[i-time_step:i, 0])
            y_train.append(label_data[i-1, 0])
    return np.array(x_train), np.array(y_train)

In [ ]:
time_step = 50

df = splited_df[0]
train_data = df['VehicleSpeed_km_h_'].values.reshape(-1, 1)
label_data = df['Battery_Status']

train_data[0:50, 0]


# train_data.shape

array([ 0.        , 11.17999935, 11.17999935, 11.17999935, 11.17999935,
       18.93000031, 26.02999878, 26.02999878, 26.02999878, 31.9299984 ,
       37.31999969, 37.31999969, 37.31999969, 41.81999969, 41.81999969,
       41.81999969, 45.93999863, 45.93999863, 49.38999939, 49.38999939,
       52.31999969, 52.31999969, 52.31999969, 55.48999786, 55.48999786,
       55.48999786, 57.73999786, 57.73999786, 59.23999786, 59.23999786,
       59.23999786, 60.77000046, 60.77000046, 61.86999893, 61.86999893,
       63.43000031, 63.43000031, 63.43000031, 63.8599968 , 63.8599968 ,
       64.22000122, 64.22000122, 64.12999725, 64.12999725, 64.12999725,
       64.12999725, 64.30999756, 64.30999756, 64.30999756, 64.30999756])

In [ ]:
time_step = 50

x_train, y_train = generate_xy(splited_df, time_step)

In [ ]:
x_train.shape

(100285, 50)

In [ ]:
y_train.shape

(100285,)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train_sc = sc.fit_transform(x_train)

x_train_sc.shape

(100285, 50)

In [ ]:
x_train = np.reshape(x_train_sc, (x_train_sc.shape[0], x_train_sc.shape[1], 1))
x_train.shape

(100285, 50, 1)

In [ ]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units=100, return_sequences= True, input_shape=(50, 1)))
# regressor.add(Dropout(0.2))

regressor.add(LSTM(units=100, return_sequences= True))
# regressor.add(Dropout(0.2))

regressor.add(LSTM(units=100, return_sequences= True))
# regressor.add(Dropout(0.2))

regressor.add(LSTM(units=50))
# regressor.add(Dropout(0.2))

regressor.add(Dense(units=2))

In [ ]:
regressor.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_28 (LSTM)               (None, 50, 100)           40800     
_________________________________________________________________
lstm_29 (LSTM)               (None, 50, 100)           80400     
_________________________________________________________________
lstm_30 (LSTM)               (None, 50, 100)           80400     
_________________________________________________________________
lstm_31 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 102       
Total params: 231,902
Trainable params: 231,902
Non-trainable params: 0
_________________________________________________________________


In [ ]:
regressor.compile(optimizer='adam', metrics=['accuracy'], loss='mse')

In [ ]:
import tensorflow as tf
tf.random.set_seed(1234)

regressor.fit(x_train, y_train, epochs=10, batch_size=10000)

Epoch 1/10
11/11 [==============================] - 17s 1s/step - loss: 0.2147 - accuracy: 0.5818
Epoch 2/10
11/11 [==============================] - 11s 1s/step - loss: 0.1795 - accuracy: 0.4175
Epoch 3/10
11/11 [==============================] - 11s 1s/step - loss: 0.1675 - accuracy: 0.6087
Epoch 4/10
11/11 [==============================] - 11s 1s/step - loss: 0.1624 - accuracy: 0.3622
Epoch 5/10
11/11 [==============================] - 11s 1s/step - loss: 0.1531 - accuracy: 0.6206
Epoch 6/10
11/11 [==============================] - 11s 999ms/step - loss: 0.1450 - accuracy: 0.4140
Epoch 7/10
11/11 [==============================] - 11s 1s/step - loss: 0.1386 - accuracy: 0.5547
Epoch 8/10
11/11 [==============================] - 11s 999ms/step - loss: 0.1315 - accuracy: 0.4835
Epoch 9/10
11/11 [==============================] - 11s 1s/step - loss: 0.1228 - accuracy: 0.4867
Epoch 10/10
11/11 [==============================] - 11s 998ms/step - loss: 0.1191 - accuracy: 0.4719
